In [1]:
from pprint import pprint
from io import StringIO
import logging

import numpy as np
import requests

from neuron_morphology.swc_io import morphology_from_swc
from neuron_morphology.feature_extractor.feature_extractor import FeatureExtractor
from neuron_morphology.feature_extractor.marked_feature import specialize
from neuron_morphology.feature_extractor.__main__ import unnest
from neuron_morphology.feature_extractor.data import Data
from neuron_morphology.constants import AXON, APICAL_DENDRITE, BASAL_DENDRITE, SOMA
from neuron_morphology.feature_extractor.feature_specialization import NEURITE_SPECIALIZATIONS

In [2]:
logging.getLogger().setLevel(logging.INFO)

morphology = morphology_from_swc(StringIO(requests.get(
    "http://celltypes.brain-map.org/api/v2/well_known_file_download/491120375"
).text))

## Layer Histograms

These are histograms of node (or other point feature) depths within cortical layers. The bins for the histograms are defined in terms of reference layer boundary depths 

In [3]:
# we provide a few collections of reference layer boundary depths (in microns). You can define your own by using 
# neuron_morphology.features.layer.reference_layer_depths.ReferenceLayerDepths

from neuron_morphology.features.layer.reference_layer_depths import WELL_KNOWN_REFERENCE_LAYER_DEPTHS
pprint(WELL_KNOWN_REFERENCE_LAYER_DEPTHS)

{'human_me_met': {'1': ReferenceLayerDepths(pia_side=0.0, wm_side=247.9338740595, scale=True),
                  '2': ReferenceLayerDepths(pia_side=247.9338740595, wm_side=410.0775480852, scale=True),
                  '3': ReferenceLayerDepths(pia_side=410.0775480852, wm_side=1322.3048788711, scale=True),
                  '4': ReferenceLayerDepths(pia_side=1322.3048788711, wm_side=1557.8034331092, scale=True),
                  '5': ReferenceLayerDepths(pia_side=1557.8034331092, wm_side=2189.3775739155, scale=True),
                  '6': ReferenceLayerDepths(pia_side=2189.3775739155, wm_side=3077.3662871205, scale=True),
                  'wm': ReferenceLayerDepths(pia_side=3077.3662871205, wm_side=3277.3662871205, scale=False)},
 'human_mtg': {'1': ReferenceLayerDepths(pia_side=0.0, wm_side=247.2137476577, scale=True),
               '2': ReferenceLayerDepths(pia_side=247.2137476577, wm_side=407.2001623338, scale=True),
               '3': ReferenceLayerDepths(pia_side=407.20016233

In [10]:
from neuron_morphology.features.layer.layer import normalized_depth_histogram
from neuron_morphology.features.layer.layered_point_depths import LayeredPointDepths

data = Data(
    morphology,  # this is not actually used!
    reference_layer_depths=WELL_KNOWN_REFERENCE_LAYER_DEPTHS["mouse"],
    layered_point_depths=LayeredPointDepths(
        ids=[0, 1, 2, 3],
        layer_names=["1", "2/3", "2/3", "wm"],
        depths=[100.0, 200.0, 300.0, 1100.0],
        point_types=[APICAL_DENDRITE, APICAL_DENDRITE, APICAL_DENDRITE, AXON],
        local_layer_pia_side_depths=[1.0, 117.0, 118.0, 958.0],
        local_layer_wm_side_depths=[117.0, 350.0, 351.0, np.nan]
    )
)

run = (
    FeatureExtractor()
        .register_features([
            specialize(normalized_depth_histogram, NEURITE_SPECIALIZATIONS)
        ])
        .extract(data)
)

for feature_name, result in unnest(run.results).items():
    found = np.nonzero(result.counts)[0]
    print(feature_name, result.bin_edges[found], result.bin_edges[found+1])
    

INFO:root:skipping mark (validation failed): type
INFO:root:selected marks: <bound method FeatureExtractionRun.select_marks of <neuron_morphology.feature_extractor.feature_extraction_run.FeatureExtractionRun object at 0x7f10c49c2b90>>
INFO:root:skipping feature: apical_dendrite.normalized_depth_histogram. Found extra marks: ['RequiresApical']
INFO:root:selected features: ['basal_dendrite.normalized_depth_histogram', 'axon.normalized_depth_histogram', 'all_neurites.normalized_depth_histogram', 'dendrite.normalized_depth_histogram']


axon.normalized_depth_histogram.wm [1097.05921309] [1102.05921309]
all_neurites.normalized_depth_histogram.1 [100.] [105.]
all_neurites.normalized_depth_histogram.2/3 [196.84067155 296.84067155] [201.84067155 301.84067155]
all_neurites.normalized_depth_histogram.wm [1097.05921309] [1102.05921309]
dendrite.normalized_depth_histogram.1 [100.] [105.]
dendrite.normalized_depth_histogram.2/3 [196.84067155 296.84067155] [201.84067155 301.84067155]
